# Creating a reproducible pipeline using TFX

Parts copied from: https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/penguin_simple.ipynb

In [1]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.6.0
TFX version: 1.3.0


In [2]:
from absl import logging
logging.set_verbosity(logging.WARNING)

In [3]:
DATA_DIR = "data"

In [4]:
!mkdir {DATA_DIR}

In [5]:
!curl -o {DATA_DIR}/data.csv https://raw.githubusercontent.com/embarced/notebooks/master/mlops/insurance-customers-risk-1500.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 54421  100 54421    0     0   492k      0 --:--:-- --:--:-- --:--:--  492k


In [6]:
!ls -l data

total 56
-rw-r--r-- 1 olli olli 54421 Oct 26 17:11 data.csv


In [7]:
!head {DATA_DIR}/data.csv

speed,age,miles,group,risk
97.0,44.0,30.0,1,0.5976112279191053
135.0,63.0,29.0,1,0.4527103520003165
111.0,26.0,34.0,0,0.750233962021037
97.0,25.0,10.0,1,0.32524900971290915
114.0,38.0,22.0,2,0.26973096398543817
130.0,55.0,34.0,0,0.5871633471963134
118.0,40.0,51.0,0,0.8753213424169751
143.0,42.0,34.0,2,0.23665405507569381
110.0,43.0,31.0,2,0.0


In [8]:
!ls -l

total 12724
-rw-r--r-- 1 olli olli 1431909 Oct 24 18:13 1_mlops_train.ipynb
-rw-r--r-- 1 olli olli  175325 Oct 26 16:55 2_mlops_serve.ipynb
-rw-r--r-- 1 olli olli 1686560 Oct 26 17:09 3_mlops_shift.ipynb
-rw-r--r-- 1 olli olli      50 Oct 26 16:56 3_mlops_shift.ipynb:Zone.Identifier
-rw-r--r-- 1 olli olli   36115 Oct 26 16:57 4_mlops_tfx.ipynb
-rw-r--r-- 1 olli olli      50 Oct 26 16:57 4_mlops_tfx.ipynb:Zone.Identifier
-rw-r--r-- 1 olli olli   39190 Oct 26 16:57 5_mlops_tfx_ext.ipynb
-rw-r--r-- 1 olli olli      50 Oct 26 16:57 5_mlops_tfx_ext.ipynb:Zone.Identifier
drwxr-xr-x 4 olli olli    4096 Oct 24 17:34 classifier
-rw-r--r-- 1 olli olli 3130024 Oct 26 17:09 classifier.h5
-rw-r--r-- 1 olli olli 2892775 Oct 24 17:34 classifier.tgz
drwxr-xr-x 2 olli olli    4096 Oct 26 17:11 data
-rw-r--r-- 1 olli olli 2110237 Aug 14 20:29 generate.ipynb
drwxr-xr-x 3 olli olli    4096 Oct 24 18:42 insurance
-rw-r--r-- 1 olli olli   54500 Aug 14 20:29 insurance-customers-risk-1500-shift.csv
-rw-r--r--

In [9]:
_trainer_module_file = 'trainer.py'

In [10]:
%%writefile {_trainer_module_file}

from tensorflow.keras.layers import InputLayer, Dense, Dropout, \
                                    BatchNormalization, Activation,\
                                    Input, concatenate
from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = ['age', 'speed']
_LABEL_KEY = 'group'

_TRAIN_BATCH_SIZE = 32
_EVAL_BATCH_SIZE = 32

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}

num_features = len(_FEATURE_KEYS)
dropout = 0.5

def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """

  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = Dense(500)(d)
    d = Activation('relu')(d)
    d = BatchNormalization()(d)
    d = Dropout(dropout)(d)

  outputs = Dense(name='output', units=3, activation='softmax')(d)

  model = keras.Model(inputs=inputs, outputs=outputs)

  model.compile(
      optimizer=keras.optimizers.Adam(),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Writing trainer.py


In [ ]:
# !cat trainer.py

In [11]:
# train:eval 2:1
# epoch = 1
# examples = 500
epoch = 100
examples = 1000
steps = (epoch * examples)/128
steps

781.25

In [12]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=5000),
      eval_args=tfx.proto.EvalArgs(num_steps=15))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

In [13]:
pipeline = _create_pipeline(
      pipeline_name='insurance-basic',
      pipeline_root='pipeline',
      data_root=DATA_DIR,
      module_file=_trainer_module_file,
      serving_model_dir='model',
      metadata_path='metadata.db')
pipeline

In [20]:
%%time 

tfx.orchestration.LocalDagRunner().run(pipeline)

/home/olli/anaconda3/envs/tfx/lib/python3.8/site-packages/keras/backend.py:4906: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


5000/5000 [==============================] - 30s 6ms/step - loss: 0.8414 - sparse_categorical_accuracy: 0.6320 - val_loss: 0.6679 - val_sparse_categorical_accuracy: 0.7271
INFO:tensorflow:Assets written to: pipeline/Trainer/model/5/Format-Serving/assets


INFO:tensorflow:Assets written to: pipeline/Trainer/model/5/Format-Serving/assets


CPU times: user 39.8 s, sys: 18.1 s, total: 57.9 s
Wall time: 36.4 s


In [15]:
!ls -l pipeline

total 16
drwxr-xr-x 4 olli olli 4096 Oct 26 17:17 CsvExampleGen
drwxr-xr-x 4 olli olli 4096 Oct 26 17:17 Pusher
drwxr-xr-x 5 olli olli 4096 Oct 26 17:17 Trainer
drwxr-xr-x 2 olli olli 4096 Oct 26 17:17 _wheels


In [16]:
!find pipeline

pipeline
pipeline/Trainer
pipeline/Trainer/model_run
pipeline/Trainer/model_run/2
pipeline/Trainer/model
pipeline/Trainer/model/2
pipeline/Trainer/model/2/Format-Serving
pipeline/Trainer/model/2/Format-Serving/keras_metadata.pb
pipeline/Trainer/model/2/Format-Serving/saved_model.pb
pipeline/Trainer/model/2/Format-Serving/assets
pipeline/Trainer/model/2/Format-Serving/variables
pipeline/Trainer/model/2/Format-Serving/variables/variables.index
pipeline/Trainer/model/2/Format-Serving/variables/variables.data-00000-of-00001
pipeline/Trainer/.system
pipeline/Trainer/.system/executor_execution
pipeline/Trainer/.system/executor_execution/2
pipeline/CsvExampleGen
pipeline/CsvExampleGen/examples
pipeline/CsvExampleGen/examples/1
pipeline/CsvExampleGen/examples/1/Split-eval
pipeline/CsvExampleGen/examples/1/Split-eval/data_tfrecord-00000-of-00001.gz
pipeline/CsvExampleGen/examples/1/Split-train
pipeline/CsvExampleGen/examples/1/Split-train/data_tfrecord-00000-of-00001.gz
pipeline/CsvExampleGen/.

In [17]:
# List files in created model directory.
!find model

model
model/1635261467
model/1635261467/keras_metadata.pb
model/1635261467/saved_model.pb
model/1635261467/assets
model/1635261467/variables
model/1635261467/variables/variables.index
model/1635261467/variables/variables.data-00000-of-00001


In [18]:
!ls -l

total 12884
-rw-r--r-- 1 olli olli 1431909 Oct 24 18:13 1_mlops_train.ipynb
-rw-r--r-- 1 olli olli  175325 Oct 26 16:55 2_mlops_serve.ipynb
-rw-r--r-- 1 olli olli 1686560 Oct 26 17:09 3_mlops_shift.ipynb
-rw-r--r-- 1 olli olli   23739 Oct 26 17:22 4_mlops_tfx.ipynb
-rw-r--r-- 1 olli olli   39190 Oct 26 16:57 5_mlops_tfx_ext.ipynb
drwxr-xr-x 2 olli olli    4096 Oct 26 17:17 __pycache__
drwxr-xr-x 4 olli olli    4096 Oct 24 17:34 classifier
-rw-r--r-- 1 olli olli 3130024 Oct 26 17:09 classifier.h5
-rw-r--r-- 1 olli olli 2892775 Oct 24 17:34 classifier.tgz
drwxr-xr-x 2 olli olli    4096 Oct 26 17:11 data
-rw-r--r-- 1 olli olli 2110237 Aug 14 20:29 generate.ipynb
drwxr-xr-x 3 olli olli    4096 Oct 24 18:42 insurance
-rw-r--r-- 1 olli olli   54500 Aug 14 20:29 insurance-customers-risk-1500-shift.csv
-rw-r--r-- 1 olli olli   54435 Aug 14 20:29 insurance-customers-risk-1500-test.csv
-rw-r--r-- 1 olli olli   54421 Aug 14 20:29 insurance-customers-risk-1500.csv
-rw-r--r-- 1 olli olli  172032 Oc